# 💻 INITIALISATION

* Lunch this part to initialize the code

* it comports : modules and functions

## ⚙️  & Import python modules

1)  modules with !pip and !apt-get commands

2) import those modules

In [1]:
import time
import subprocess
from openpyxl import load_workbook
import re
from selenium import webdriver
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import  WebDriverException
import requests
import os
from IPython.display import HTML
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from urllib3.exceptions import MaxRetryError
from selenium.common.exceptions import InvalidSessionIdException
from fuzzywuzzy import fuzz
import shutil
from schwifty import IBAN
from schwifty.exceptions import InvalidChecksumDigits
import PyPDF2
from pdf2image import convert_from_path
import cv2
import pytesseract


## 📈 FUNCTIONS

### 1) KBIS function

This function use regex to fin SIREN and Adress from the french KBIS

### 2) Orbis function

This function scrape the ORBIS website with the general URL gave to your company

### 3) VAT VIES function

This function use the VAT number from orbis to scrape the traget company VAT state.

### 4) INSEE function

This function use the SIRET number from orbis to scrape the traget company INSEE informations (SIRET, SIREN, adress, activity)


### HOW TO USE THIS PART ?

* just lunch it to initial (only few seconds)

#### Fonction kbis

In [2]:
def extract_kbis(pdf_file_name) :

    pdf_file = open(pdf_file_name, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""

    # Concatenate the text of all pages in the PDF file
    for page in pdf_reader.pages:
        text += page.extract_text()


    # Extract the SIREN number
    siren_pattern = r"\d{3} \d{3} \d{3}"
    siren_match = re.search(siren_pattern, text)

    if siren_match:
        siren = siren_match.group()
        siren = siren.replace(" ", "")
    else:
        siren = "SIREN non trouvé."

    # Extract the address of the head office
    adresse_pattern = r"Adresse du siège (.+?)\n"
    adresse_match = re.search(adresse_pattern, text)

    if adresse_match:
        adresse = adresse_match.group(1)
        adresse = adresse.replace("\n", " ")
    else:
        adresse = ""

    # Extraire la dénomination ou la raison sociale
    raison_sociale_pattern = r"raison sociale\s*(.+)\s*"
    raison_sociale_match = re.search(raison_sociale_pattern, text)
    if raison_sociale_match:
        raison_sociale = raison_sociale_match.group(1)
        raison_sociale = raison_sociale.strip()
    else:
        raison_sociale = ""

    # Return the extracted data
    list = [siren.replace(" ", ""), adresse, raison_sociale, ""]
    list = [element.upper() for element in list]

    return list

#extract_kbis(r'LINK_TO_TMP_FOLDER\kbis.pdf')

In [3]:
def ocr_kbis(path):

  endpoint = "YOUR_AZURE_ENDPOINT"
  key = "YOUR_AZURE_API_KEY"

  model_id = "NAME OF YOUR MODEL"
  #path = '/content/kbis.pdf'

    # sample document
  with open(path, 'rb') as document:
      document_analysis_client = DocumentAnalysisClient(
          endpoint=endpoint, credential=AzureKeyCredential(key)
      )

  # Make sure your document's type is included in the list of document types the custom model can analyze
      poller = document_analysis_client.begin_analyze_document(model_id, document)
      result = poller.result()
      result_list = []

      # Your existing code to obtain the 'result' JSON
  for idx, document in enumerate(result.documents):
      print("--------Analyzing kbis #{}--------".format(idx + 1))

      

      for name, field in document.fields.items():
        
        if field.value is None :

          field_value = field.content

        else :

          field_value = field.value

        result_list.append(field_value)


  result_list[0] = result_list[0].replace(" ", "").upper()
  result_list = [element.upper() for element in result_list if element is not None]

  return result_list

#### Fonction orbis

In [31]:
def extract_orbis(siren, driver):
    
    liste_resultat = []

    username = 'your_bvd_username'
    password = 'your_bvd_password'
    driver.get(f"https://orbis.bvdinfo.com/orbis/Companies/Report/BVDId/FR{siren}?user={username}&pw={password}&amp;restart=1")

    wait = WebDriverWait(driver, 10)
    #driver.save_screenshot('screenie.png')
    time.sleep(3)


    #'try ... except' loops to bypass walk-me divs
    try:

      # Code A

      raison_sociale = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/table[1]/tbody/tr[1]/td[2]/span').text
      activite = driver.find_element(By.XPATH, "//div[@id=\'main-content\']/div[2]/table/tbody/tr[2]/td/span").text
      adresse = driver.find_element(By.XPATH, "//*[@id=\"flowList1\"]/li/table/tbody/tr/td/table/tbody/tr[1]/td[1]/table/tbody").text


    except (ElementClickInterceptedException, NoSuchElementException):

      try :

        driver.find_element(By.XPATH, '//*[@id="wm-shoutout-389935"]/div[4]/div[2]/span[2]').click()
        raison_sociale = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/table[1]/tbody/tr[1]/td[2]/span').text
        activite = driver.find_element(By.XPATH, "//div[@id=\'main-content\']/div[2]/table/tbody/tr[2]/td/span").text
        adresse = driver.find_element(By.XPATH, "//*[@id=\"flowList1\"]/li/table/tbody/tr/td/table/tbody/tr[1]/td[1]/table/tbody").text

      except (ElementClickInterceptedException, NoSuchElementException):

        try :

          driver.find_element(By.CSS_SELECTOR, '[id^="wm-shoutout-"] > div.wm-content > div.buttons-wrapper > span').click()
          raison_sociale = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/table[1]/tbody/tr[1]/td[2]/span').text
          activite = driver.find_element(By.XPATH, "//div[@id=\'main-content\']/div[2]/table/tbody/tr[2]/td/span").text
          adresse = driver.find_element(By.XPATH, "//*[@id=\"flowList1\"]/li/table/tbody/tr/td/table/tbody/tr[1]/td[1]/table/tbody").text


        except (ElementClickInterceptedException, NoSuchElementException):

          try:
              driver.find_element(By.XPATH, '//*[@title="Close Survey"]').click()
              raison_sociale = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/table[1]/tbody/tr[1]/td[2]/span').text
              activite = driver.find_element(By.XPATH, "//div[@id=\'main-content\']/div[2]/table/tbody/tr[2]/td/span").text
              adresse = driver.find_element(By.XPATH, "//*[@id=\"flowList1\"]/li/table/tbody/tr/td/table/tbody/tr[1]/td[1]/table/tbody").text

          except :
             
            driver.find_element(By.CSS_SELECTOR, 'button[class*="wm"]').click()
            raison_sociale = driver.find_element(By.XPATH, '//*[@id="main-content"]/div[2]/table[1]/tbody/tr[1]/td[2]/span').text
            activite = driver.find_element(By.XPATH, "//div[@id=\'main-content\']/div[2]/table/tbody/tr[2]/td/span").text
            adresse = driver.find_element(By.XPATH, "//*[@id=\"flowList1\"]/li/table/tbody/tr/td/table/tbody/tr[1]/td[1]/table/tbody").text

             




    #avoir les informations clefs

    #verifier l'icone drapeau
    try:
        # trouver l'élément avec le XPath pour le drapeau vert
        drapeau = driver.find_element(By.XPATH, '//*[@id="flowListCompanyStatus"]/li[2]/table/tbody/tr[2]/td[1]/img[contains(@src,"flag-green_40px.png")]')
        drapeau = 'drapeau vert'
        print("L'icône drapeau vert est présente sur la page.")

    except NoSuchElementException:
        
        try:
            # trouver l'élément avec le XPath pour le drapeau orange
            drapeau = driver.find_element(By.XPATH, '//*[@id="flowListCompanyStatus"]/li[2]/table/tbody/tr[2]/td[1]/a/img[contains(@src,"flag-yellow_40px.png")]')
            drapeau = 'drapeau orange'
            print("ATTENTION : DRAPEAU ORANGE !!")

        except NoSuchElementException:

            drapeau = None
            print("Aucun drapeau n'a été trouvé sur la page.")

    #rentrer dans le menu profil entreprise

    driver.find_element(By.LINK_TEXT, "Profil d\'entreprise").click()
    driver.find_element(By.LINK_TEXT, "Identifiants").click()

    #avoir les identifiants

    time.sleep(3)
    page_text = driver.page_source

    adresse = adresse.replace("\n", " ")

    siren = re.compile(r"SIREN number.*?(\d{9})")
    siren = siren.search(page_text).group(1)

    siret = re.compile(r"\d{9}-\d{5}")
    siret = siret.search(page_text).group(0)
    siret = siret.replace("-", "")
  
    tva_pattern = r"\bFR\d{2} \d{9}\b"
    tva_match = re.search(tva_pattern, page_text)

    if tva_match:
        
        tva = tva_match.group()
        tva = tva.replace(" ", "")

    else:

        tva = "WARNING : la TVA n'existe pas"
        tva = ["",  "", "", "", "", "", ""]
        return tva
      
    
    #print(cate)
    if "Active" == activite :

      liste_resultat = [raison_sociale,  siren, siret, tva, adresse, activite, drapeau]
      liste_resultat = [element.upper() for element in liste_resultat]
      return liste_resultat

    else :

      print("L'activité est pour le moins étrange...")
      liste_resultat = [raison_sociale,  siren, siret, tva, adresse, 'ATTENTION ACTIVITE', drapeau]
      liste_resultat = [element.upper() for element in list]
      return liste_resultat


#driver = initier_driver()
#appel = extract_orbis(491325528, driver)
#print(appel)


#### Fonction INSEE

In [5]:
def api_insee_siret(siret):

  url = f'https://api.insee.fr/entreprises/sirene/V3/siret/{siret}'
  headers = {
      'Accept': 'application/json',
      'Authorization': 'Bearer {YOUR_API_KEY}'
  }

  response = requests.get(url, headers=headers, verify=False)

  if response.status_code == 200:
      data = response.json()
      # Process the JSON response as needed
  else:

      return f"'Request failed with status code:', {response.status_code}, veuillez verifier le siret sur INSEE"


  adresse = data['etablissement']['adresseEtablissement']
  siren = data['etablissement']['siren']
  company_name = data['etablissement']['uniteLegale']['denominationUniteLegale']
  activity = data['etablissement']['periodesEtablissement'][0]['etatAdministratifEtablissement']
  liste = [company_name, siren]

  numero_voie = adresse["numeroVoieEtablissement"]
  type_voie = adresse["typeVoieEtablissement"]
  libelle_voie = adresse["libelleVoieEtablissement"]
  code_postal = adresse["codePostalEtablissement"]
  libelle_commune = adresse["libelleCommuneEtablissement"]

  adresse_complete = f"{numero_voie} {type_voie} {libelle_voie} {code_postal} {libelle_commune}"

  liste = [company_name, siren, siret, numero_voie, type_voie, libelle_voie, code_postal, libelle_commune]


  if activity == 'A' :


    liste.append('active')
    return liste

  else :

    colored_text = '<span style="color: red;">fait gaffe à l activite</span>'
    HTML(colored_text)
    return "fait gaffe a l'activité"


#insee = api_insee_siret(79816678100046)
#print(insee)

#### Fonction TVA

In [6]:
def extract_tva(num_tva, driver):


    #driver = webdriver.Chrome()
    driver.get("https://ec.europa.eu/taxation_customs/vies/vatRequest.html")
    wait = WebDriverWait(driver, 10)

    # Récupérer la hauteur totale de la page
    total_height = driver.execute_script("return document.body.scrollHeight")

    # Calculer la position à laquelle faire défiler la page
    scroll_pos = int(total_height * 0.90)
    wait.until(EC.visibility_of_element_located((By.ID, "select-country")))

     # Faire défiler la page jusqu'à la position calculée
    driver.execute_script("window.scrollTo(0, {});".format(scroll_pos))

    driver.find_element(By.XPATH, '//*[@id="select-country"]').click()
    dropdown = driver.find_element(By.XPATH, '//*[@id="select-country"]')

    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="select-country"]/option[13]')))
    dropdown.find_element(By.XPATH, '//*[@id="select-country"]/option[13]').click()

    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="select-country"]/option[13]')))

    driver.find_element(By.XPATH, '//*[@id="vat-validation-form"]/div/div[2]/input').click()
    driver.find_element(By.XPATH, '//*[@id="vat-validation-form"]/div/div[2]/input').send_keys(f'{num_tva}')
    element = driver.find_element(By.XPATH, '//*[@id="vat-validation-form"]/div/div[5]/button/span/span')
    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="vat-validation-form"]/div/div[5]/button/span/span')))
    driver.execute_script("arguments[0].scrollIntoView();", element)
    element.click()


    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".text-result")))
    result = driver.find_element(By.CSS_SELECTOR, ".text-result").text

    if result == "Oui, numéro de TVA valide":
        return "TVA VALIDE"

    elif result == "Yes, valid VAT number":
        return "TVA VALIDE"

    else:
        return " ATTENTION TVA INVALIDE"


#driver = initier_driver()
#tva = extract_tva('30351363494', driver)

#### Fonction RIB

In [7]:
def detecter_bic(texte):
    match = re.search(r'\b\w+XXX', texte)
    if match:
        return match.group()
    else:
        return None

def extract_rib(file_path):
    if os.path.splitext(file_path)[1].lower() == '.pdf':
      # Ouvrir le fichier PDF
      with open(file_path, 'rb') as pdf_file:
          # Créer un objet PyPDF2 pour le fichier PDF

          pdf_reader = PyPDF2.PdfReader(pdf_file)
          text = ""

          # Concatenate the text of all pages in the PDF file
          for page in pdf_reader.pages:
              text += page.extract_text()
          iban_match = re.search(r'FR\d{2}\s?\w{4}\s?\w{4}\s?\w{4}\s?\w{4}\s?\w{4}\s?\w{2,3}', text)
          if (text.replace(" ", "").replace("\n", "") != "") and iban_match :
              text = text

          else :

                  pages = convert_from_path(file_path, dpi=200)

                  for i, page in enumerate(pages):
                          page.save(f'page_{i+1}.jpg', 'JPEG')
                          images = [f'page_{i+1}.jpg' for i in range(len(pages))]
                  else:
                      images = [file_path]

                  # Extraire le texte de chaque image en utilisant la bibliothèque Tesseract OCR

                      # Charger l'image
                  image = cv2.imread('/content/page_1.jpg')

                      # Appliquer un prétraitement à l'image pour améliorer la qualité de l'OCR
                  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                  gray = cv2.medianBlur(gray, 3)
                  gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

                      # Extraire le texte de l'image
                  text = pytesseract.image_to_string(gray, lang='eng')

    else :

      image = cv2.imread(file_path)

      # Appliquer un prétraitement à l'image pour améliorer la qualité de l'OCR
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      gray = cv2.medianBlur(gray, 3)
      gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

      # Extraire le texte de l'image
      text = pytesseract.image_to_string(gray, lang='eng')

    # Utiliser une expression régulière pour rechercher un IBAN
    iban_match = re.search(r'FR\d{2}\s?\w{4}\s?\w{4}\s?\w{4}\s?\w{4}\s?\w{4}\s?\w{2,3}', text)

    #initialiser liste resultat

    liste_resultat = []

    # Si un IBAN est trouvé, vérifier sa validité avec Schwifty
    if iban_match:
        iban_match = iban_match.group().replace(' ', '')  # Supprimer les espaces éventuels dans l'IBAN
        iban = IBAN(iban_match)
        #print(iban_match)
        #print(iban.country_code)
        #print(iban.bank_code)
        #print(iban.account_code)
        #print(iban.bic)
        bic_verif = detecter_bic(text)
        if bic_verif == None :
          liste_resultat.append(str(iban.bic))
        else :
          liste_resultat.append(bic_verif)

        if iban.validate(validate_bban=True):
            liste_resultat.append(str(iban))
            return liste_resultat
        else :
            return "iban invalide"

    else :

      print("iban non trouvé veuillez rentrer les numéros à la main")
      bic = input("Veuillez rentrer le BIC")
      iban = input("Veuillez rentrer l'iban")
      liste_resultat = [bic, iban]
      return liste_resultat

In [8]:
def ocr_rib(path) :

  endpoint = "YOUR_AZURE_ENDPOINT"
  key = "YOUR_API_KEY"

  model_id = "NAME OF YOUR MODEL"
  #path = '/content/kbis.pdf'

    # sample document
  with open(path, 'rb') as document:
      document_analysis_client = DocumentAnalysisClient(
          endpoint=endpoint, credential=AzureKeyCredential(key)
      )

  # Make sure your document's type is included in the list of document types the custom model can analyze
      poller = document_analysis_client.begin_analyze_document(model_id, document)
      result = poller.result()
      result_list = []

      # Your existing code to obtain the 'result' JSON
  for idx, document in enumerate(result.documents):
      print("--------Analyzing rib #{}--------".format(idx + 1))

      for name, field in document.fields.items():

          field_value = field.value if field.value else field.content
          if field_value is not None :
            result_list.append(field_value)

  i = 0
  for elem in result_list :
     if "DocumentField(value_type=dictionary" in str(elem[0]) :
        position_iban = i
        break
     
     else :
        
        i = i+1
  

  try :
    
    iban = result_list[position_iban][0].value.get('IBAN').value.replace(" ", "")

  except :

    iban = ""

  try :

    bic = result_list[position_iban][0].value.get('SWIFT').value.replace(" ", "")

  except :

    bic = ""

  liste = []

  liste.append(iban)
  liste.append(bic)

  return liste




#### Text file generation

In [9]:
def fichier_texte(list_orbis, list_insee, list_rib, list_chercher_pj, num_bp):

    nom = list_orbis[0]
    siren = list_orbis[1]
    siret = list_orbis[2]
    tva = list_orbis[3]
    adresse = list_orbis[4]
    numero_rue = list_insee[3]

    if numero_rue == None :
      numero_rue = ''
    else :
      numero_rue = numero_rue

    rue = str(list_insee[4]) + " " + str(list_insee[5])
    postal_code = list_insee[6]
    city = list_insee[7]
    try :
      departement = postal_code[:2]
    except :
      departement = ""

    bic = list_rib[1]
    iban = list_rib[0]
    cate_achat = list_chercher_pj[0]
    entites = []

    for element in list_chercher_pj[1]:
        entites.append(element)

    


    # Enregistrer les valeurs dans un fichier texte
    with open("texte.txt", "w") as f :
        f.write(f"{nom}\n")
        f.write(f"{rue}\n")
        f.write(f"{numero_rue}\n")
        f.write(f"{postal_code}\n")
        f.write(f"{city}\n")
        f.write(f"{departement}\n")
        f.write(f"{siren}\n")
        f.write(f"{siret}\n")
        f.write(f"{tva}\n")
        f.write(f"{bic}\n")
        f.write(f"{iban}\n")
        f.write(f"{cate_achat}\n")
        for entite in entites :
          
          f.write(f"{entite}\n")
          if entite == "G100" and entite != "G200" :
             f.write("G200\n")


        f.write(f"{num_bp}\n")

    # Afficher le contenu du fichier texte dans la console
    with open('texte.txt', 'r') as f:
        contenu = f.read()
        print(contenu)



#### Fonction Scraping YB

In [10]:
def supprimer_fichiers_dans_tmp():

    dossier_tmp = r"LINK_TO_TMP_FOLDER"  

    try:
        # Obtenez la liste des fichiers dans le dossier tmp
        fichiers_tmp = os.listdir(dossier_tmp)

        # Parcourez chaque fichier et supprimez-le
        for fichier in fichiers_tmp:
            chemin_fichier = os.path.join(dossier_tmp, fichier)
            os.remove(chemin_fichier)


        print("Tous les fichiers dans le dossier tmp ont été supprimés avec succès.")

    except Exception as e:
        print(f"Erreur lors de la suppression des fichiers : {e}")

# Utilisez la fonction pour supprimer les fichiers dans le dossier tmp
#supprimer_fichiers_dans_tmp()

In [11]:
def deplacer_fichiers_vers_archive(nom_fournisseur):

    dossier_tmp = r"LINK_TO_TMP_FOLDER"  # Remplacez cela par le chemin réel de votre dossier tmp
    dossier_archive = r"LINK_TO_ARCHIVE_FOLDER"  # Remplacez cela par le chemin réel de votre dossier archive

    try:
        # Obtenez la liste des fichiers dans le dossier tmp
        fichiers_tmp = os.listdir(dossier_tmp)

        # Créez le chemin complet du dossier dans le dossier archive
        chemin_dossier_destination = os.path.join(dossier_archive, nom_fournisseur)

        # Créez le dossier dans le dossier archive
        os.makedirs(chemin_dossier_destination)
        print(f"Dossier créé : {chemin_dossier_destination}")

        # Parcourez chaque fichier et déplacez-le vers le dossier nouvellement créé
        for fichier in fichiers_tmp:
            chemin_fichier_source = os.path.join(dossier_tmp, fichier)
            chemin_fichier_destination = os.path.join(chemin_dossier_destination, fichier)
            shutil.move(chemin_fichier_source, chemin_fichier_destination)

        print(f"Tous les fichiers ont été déplacés vers le dossier {nom_fournisseur} dans le dossier archive avec succès.")

    except Exception as e:
        print(f"Erreur lors du déplacement des fichiers : {e}")

# Utilisez la fonction pour déplacer les fichiers du dossier tmp vers un dossier variable dans le dossier archive
#deplacer_fichiers_vers_archive("nom_fournisseur_123")


In [12]:
def initier_driver():
    download_directory = r"LINK_TO_TMP_FOLDER"
    options = Options()
    options.add_experimental_option("prefs", {
        "download.default_directory": download_directory,
        "download.prompt_for_download": False,
        "plugins.always_open_pdf_externally": True
    })

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver

#driver = initier_driver()

In [13]:
def connexion(driver, username, password) :

    driver.maximize_window()
    wait = WebDriverWait(driver, 120)
    username = wait.until(EC.presence_of_element_located((By.XPATH,"//input[@type='email']" ))).send_keys(f'{username}')
    button = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@type='submit']"))).click()
    time.sleep(2)
    password = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@type='password']" ))).send_keys(f'{password}')
    button = driver.find_element(By.ID, "idSIButton9")
    button.click()
    if 'login' in driver.current_url :
        
        print('en attente de validation authentificator, vous avez 1 minute')
        numero_identification = wait.until(EC.presence_of_element_located((By.ID, "idRichContext_DisplaySign"))).text
        print(f"le numéro d'identification est {numero_identification}")
        driver.save_screenshot('numero_authentificator.png')

        wait.until(EC.url_contains("youbuy.ivalua.com"))

    
    print('connexion réussie')

#options = Options()
#options.add_argument("--headless")
#driver = webdriver.Chrome(options=options)
#connect = connexion(driver, userdata.get('username'), userdata.get('password'))

In [14]:
def chercher_pj(driver, numero_reg) :

    username = 'email'
    password = 'password'
    driver.get(f'https://youbuy.ivalua.com/page.aspx/fr/sup/registration_manage/{numero_reg}')
    time.sleep(4)
    url = driver.current_url
    liste = []

    if 'login' in url :

      login_microsoft = connexion(driver, username, password)

    wait = WebDriverWait(driver, 10)
    entite = driver.find_element(By.XPATH, '//*[@id="body_x_tab_prxRegistration_prxprxRegistration_x_phcContact_content"]/table/tbody/tr[4]/td[1]/div/div/div' ).text
    
    entite_finale = reclassifier_entite(entite)
    cate_achat = driver.find_element(By.XPATH, '//*[@id="body_x_tab_prxRegistration_prxprxRegistration_x_placeholder_sup_registration_170110144526_content"]/table/tbody/tr/td/div/div/div/div[1]' ).text
    cate_achat = re.search(r'\b(\d{4})\b', cate_achat).group(1)

    liste.append(cate_achat)
    liste.append(entite_finale)
    

    avancement = wait.until(EC.presence_of_element_located((By.ID, "body_x_tab_containerTab_partprxprxWorkflow")))
    avancement.click()
    time.sleep(2)
    elems = driver.find_elements(By.XPATH, "//a[@href]")
    
    for elem in elems :
        elem_href = elem.get_attribute("href")
        if '/bare.aspx/fr/fil/download/' in elem_href :

            time.sleep(1)
            try :

              driver.execute_script("arguments[0].scrollIntoView();", elem)
              elem.click()
              liste.append(elem_href)

            except :

              elem.click()
              liste.append(elem_href)

    return liste


#chercher_pj(driver, '7110')

In [15]:
def reclassifier_entite(entite):
    # Utilisation de regex pour séparer les entités
    entites = re.split(r'\n', entite)

    # Liste pour stocker les entités reclassifiées
    entites_reclassifiees = []

    # Extraire et reclasser les entités
    for entite in entites:

        if entite == "SUEZ SA":
            entites_reclassifiees.append("G100")

        elif entite == "SUEZ Holding SAS":
            entites_reclassifiees.append("G200")       

        elif entite == "DOLCE Ô SERVICE":
            entites_reclassifiees.append("D210")

        elif entite == "SUEZ SMART SOLUTIONS":
            entites_reclassifiees.append("D200")

        elif entite == "SUEZ CIRCULAR SOLUTIONS" :
            entites_reclassifiees.append("E100")

        elif entite == "ARIA Technologies" :
            entites_reclassifiees.append("D500")

        else : 
            entites_reclassifiees.append(entite)

    return entites_reclassifiees

In [16]:
def rename_file(old_path, new_name):

    # Get the directory and filename from the old path
    directory, filename = os.path.split(old_path)

    # Create the new path by combining the directory, new filename, and original extension
    new_path = os.path.join(directory, new_name)

    try:
        # Rename the file
        os.rename(old_path, new_path)
        
    except FileNotFoundError:
        print(f"Error: The file '{old_path}' does not exist.")
    except FileExistsError:
        print(f"Error: The file '{new_path}' already exists.")



In [17]:
def parcourir_dossier(path):
    # Obtenez la liste des fichiers dans le dossier
    fichiers = os.listdir(path)
    liste_chemin = []

    # Parcourez la liste des fichiers
    for fichier in fichiers:

        chemin_complet = os.path.join(path, fichier)

        # Obtenez l'extension du fichier
        extension = os.path.splitext(fichier.lower())[1][1:]
        

        # Vérifiez si le fichier a l'une des extensions spécifiées
        if extension in ('pdf', 'png', 'jpeg', 'jpg', 'gif'):
            
            # Vérifiez si 'kbis' (indépendamment de la casse) est présent dans le nom du fichier
            if 'siren' in str(fichier.lower()):
                # Renommez le fichier selon vos besoins
                nouveau_chemin = rename_file(chemin_complet, f'LINK_TO_TMP_FOLDER\\kbis.{extension}')
                liste_chemin.append(f'LINK_TO_TMP_FOLDER\\kbis.{extension}')

            elif 'situation' in str(fichier.lower()):
                # Renommez le fichier selon vos besoins
                nouveau_chemin = rename_file(chemin_complet, f'LINK_TO_TMP_FOLDER\\kbis.{extension}')
                liste_chemin.append(f'LINK_TO_TMP_FOLDER\\kbis.{extension}')

            elif 'kbis' in str(fichier.lower()):
                # Renommez le fichier selon vos besoins
                nouveau_chemin = rename_file(chemin_complet, f'LINK_TO_TMP_FOLDER\\kbis.{extension}')
                liste_chemin.append(f'LINK_TO_TMP_FOLDER\\kbis.{extension}')

            elif "rib" in str(fichier.lower()):
                # Renommez le fichier selon vos besoins
                nouveau_chemin = rename_file(chemin_complet, f'LINK_TO_TMP_FOLDER\\rib.{extension}')
                liste_chemin.append(f'LINK_TO_TMP_FOLDER\\rib.{extension}')
            
            else :

                liste_chemin.append(chemin_complet)
        

    return liste_chemin

#parcourir_dossier(r'LINK_TO_TMP_FOLDER')


In [18]:
def classify_type_pj(path):

    endpoint = "YOUR_AZURE_ENDPOINT"
    key = "YOUR_AZURE_KEY"
    model_id = "NAME OF YOUR MODEL"
    result_list = []

    try:

        with open(path, 'rb') as document:
            document_analysis_client = DocumentAnalysisClient(
                endpoint=endpoint, credential=AzureKeyCredential(key)
            )

            poller = document_analysis_client.begin_classify_document(model_id, document)
            result = poller.result()

            print("----Classified documents----")
            for doc in result.documents:
                if any(region.page_number == 1 for region in doc.bounding_regions):
                    print(
                        f"Found on {path} a document of type '{doc.doc_type or 'N/A'}' with a confidence of {doc.confidence}"
                    )

                    if doc.confidence >= 0.75:
                                
                        result_list.append(path)
                        result_list.append(doc.doc_type)

                    else :

                        result_list.append(path)
                        result_list.append('no_classification')

    except IsADirectoryError:

        result_list.append("error")
        result_list.append("error")

        pass

    return result_list


#path = r'LINK_TO_TMP_FOLDER'
#liste_dossier = parcourir_dossier(path)
#time.sleep(2)
#print(liste_dossier)
#for path in liste_dossier:
 #   if path.lower().endswith(('.pdf', '.jpeg', '.png', '.gif', '.jpg')) and not path.lower().startswith(('kbis', 'rib')):
  #      file_extension = os.path.splitext(path)[1]
   #     data = classify_type_pj(path)
    #    rename_file(data[0], f'{data[1]}{file_extension}')


In [19]:
def creer_fournisseur_youbuy(numero_reg, driver, liste_orbis, liste_insee) :

  username = 'email'
  password = 'password'
  wait = WebDriverWait(driver, 15)
  driver.get(f"https://youbuy.ivalua.com/page.aspx/fr/sup/registration_manage/{numero_reg}")
  time.sleep(3)
  url = driver.current_url
  if 'login' in url :

    login_microsoft = connexion(driver, username, password)

  else :

    print("connexion à youbuy réussie")

  valider = wait.until(EC.presence_of_element_located((By.ID, "proxyActionBar_x_valREG_SUEZ_V2TRT"))).click()
  driver.save_screenshot('SUP.png')
  a_vers_sup = wait.until(EC.presence_of_element_located((By.XPATH, "//a[starts-with(@href, '/page.aspx/fr/sup/supplier_manage/')]"))).click()

  saisir_raison_sociale = (wait.until(EC.presence_of_element_located((By.ID, "body_x_tabc_identity_prxidentity_x_txtName"))))
  saisir_raison_sociale.send_keys(Keys.CONTROL, 'a')
  saisir_raison_sociale.send_keys(f"{liste_orbis[0]}")

  saisir_siren = wait.until(EC.presence_of_element_located((By.ID, "body_x_tabc_identity_prxidentity_x_txtSupCodeNationalIdI")))
  saisir_siren.send_keys(Keys.CONTROL, 'a')
  saisir_siren.send_keys(f"{liste_orbis[1]}")

  saisir_siret = wait.until(EC.presence_of_element_located((By.ID, "body_x_tabc_identity_prxidentity_x_txtSupCodeNationalIdIi")))
  saisir_siret.send_keys(Keys.CONTROL, 'a')
  saisir_siret.send_keys(f"{liste_orbis[2]}")

  saisir_tva = wait.until(EC.presence_of_element_located((By.ID, "body_x_tabc_identity_prxidentity_x_txtTva")))
  saisir_tva.send_keys(Keys.CONTROL, 'a')
  saisir_tva.send_keys(f"{liste_orbis[3]}")

  adress = f'{liste_insee[3]} {liste_insee[4]} {liste_insee[5]}'
  saisir_adresse = wait.until(EC.presence_of_element_located((By.ID, "body_x_tabc_identity_prxidentity_x_proxyAdr_x_txtVoie")))
  saisir_adresse.send_keys(Keys.CONTROL, 'a')
  saisir_adresse.send_keys(f"{adress}")

  saisir_code_postal = wait.until(EC.presence_of_element_located((By.ID, "body_x_tabc_identity_prxidentity_x_proxyAdr_x_txtZip")))
  saisir_code_postal.send_keys(Keys.CONTROL, 'a')
  saisir_code_postal.send_keys(f"{liste_insee[6]}")

  saisir_ville = wait.until(EC.presence_of_element_located((By.ID, "body_x_tabc_identity_prxidentity_x_proxyAdr_x_txtCityfr")))
  saisir_ville.send_keys(Keys.CONTROL, 'a')
  saisir_ville.send_keys(f"{liste_insee[7]}")

  valider2 = wait.until(EC.presence_of_element_located((By.ID, "proxyActionBar_x_valSUPREF_SUEZ_V3SUP"))).click()
  valider3 = wait.until(EC.presence_of_element_located((By.ID, "proxyActionBar_x_valSUPREF_SUEZ_V3CRE"))).click()
  print("ok fournisseur créé sur YOUBUY")

In [20]:
def similaire(elem1, elem2) :

  similarite = fuzz.ratio(elem1, elem2)
  return similarite

In [21]:
def verifications_conforme(kbis, orbis, insee, tva):

  verifications = []
  adresse_insee = f'{insee[3]} {insee[4]} {insee[5]} {insee[6]} {insee[7]}'

  #verif nom

  if similaire(kbis[2].upper(), orbis[0])<25 and similaire(orbis[0], insee[0])<25 :

    verifications.append('attention nom')


  #verif siren
  if kbis[0] != orbis[1] and orbis[1] != insee[1] :

    verifications.append('attention siren')


  #verif siret
  if orbis[2] != insee[2] :

    verifications.append('attention siret')

  #verif adresse
  if similaire(kbis[2], orbis[4])<30 and similaire(orbis[4], adresse_insee)<30 :

    verifications.append('attention adresse')


  #verif drapeau
  if orbis[6] != 'DRAPEAU VERT':

    verifications.append('attention au drapeau')

  #verif tva
  if tva != 'TVA VALIDE' :

    verifications.append("attention à la TVA")

  #verifications [nom, siren, siret, adresse, siren]
  return verifications

#verification = verifications_conforme(kbis, orbis, insee, extract)
#print(verification)



In [22]:
def check_RVF_doublon(num_tva, num_siret) :

    liste_resultat = []

    # Charger le classeur Excel
    wb = load_workbook(r"LINK_TO_SAP_EXPORT\EXPORT.XLSX")

    # Sélectionner la feuille
    ws = wb.active  # ou vous pouvez spécifier le nom de la feuille, ex. : wb["NomDeVotreFeuille"]

    # Extraire les éléments de la colonne C à partir de la deuxième ligne
    liste_num_tva = [cell.value for cell in ws["D"][1:]]

    if num_tva in liste_num_tva :

        print('un doublon a été détécté : verifications des informations en cours ...')
        liste_num_siret = [cell.value for cell in ws["C"][1:]]
        liste_num_bp = [cell.value for cell in ws["A"][1:]]
        position = liste_num_tva.index(num_tva)
        num_vendor = liste_num_bp[position]
        liste_resultat.append(num_vendor)

        if  (liste_num_siret[position] is None) or (num_siret not in liste_num_siret[position]) :
            print(liste_num_siret[position])
            print("Le siret est erroné merci d'envoyer un mail")

            return ['MAJSAP', num_vendor]
        
        else :

            print('le siret est bon')
            return ['MAJSAP', num_vendor]



    else :

        print('aucun doublon détécté :)')
        return ['AUTOSAP', '']


#check = check_RVF_doublon(orbis[3], orbis[2])
#print(check)

In [23]:
def creer_fournisseur_dans_sap(check) :

    # Chemin complet du script PowerShell
    script_path = rf'Bureau\\automatisation\\{check}.ps1'

    # Commande PowerShell à exécuter
    powershell_command = f'C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe -ExecutionPolicy Bypass -File "{script_path}" -MyArguments blah'

    # Exécution de la commande PowerShell
    process = subprocess.Popen(['powershell', '-Command', powershell_command], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    return "le fournisseur à été créé dans SAPHIR, veuillez ajouter les PJ"

# 👨‍🌾 CALLING FUNCTION

* Put the kbis and rib in the virtual env. Make sure that their names are 'kbis.pdf' and 'rib.pdf'.

* Run the script

* The .txt file is generated

In [ ]:
numero_reg = '8177'

#Chercher les PJ

#supprimer fichier dans dossier tmp
supprimer_fichiers_dans_tmp()

try :

  liste_pieces_jointes = chercher_pj(driver, numero_reg)

except (NameError, MaxRetryError, InvalidSessionIdException, NoSuchWindowException, WebDriverException) :

  driver = initier_driver()
  liste_pieces_jointes = chercher_pj(driver, numero_reg)


time.sleep(2)

#premier check de classification
dossier = parcourir_dossier(r'LINK_TO_TMP_FOLDER')

time.sleep(2)

#Classifier les PJ
for path in dossier :

  try :

    if path.lower().endswith(('.pdf', '.jpeg', '.png', '.gif', '.jpg')) and not (('kbis' in path) or ('rib' in path)) :
        file_extension = os.path.splitext(path)[1]
        data = classify_type_pj(path)
        rename_file(data[0], f'{data[1]}{file_extension}')

  except :

    print('error')
    pass

#Analyser les PJ
#Voir présence matrice

dossier = parcourir_dossier(r'LINK_TO_TMP_FOLDER')
##analyser le kbis
try :

  position_kbis = dossier.index(next(element for element in dossier if 'kbis' in element))

except :

  kbis = print('pas de kbis trouvé')

try :

  kbis = ocr_kbis(dossier[position_kbis])

except :

  try :

    kbis = extract_kbis(dossier[position_kbis])

  except :

    path = dossier[position_kbis]
    file_extension = os.path.splitext(path)[1]
    rename_file(dossier[position_kbis], f'LINK_TO_TMP_FOLDER\\kbis_{numero_reg}.{file_extension}')
    shutil.move(f'LINK_TO_TMP_FOLDER\\kbis_{numero_reg}.{file_extension}', r"LINK_TO_TRAINING_DATAS\kbis" )
  
print(kbis)

#appel orbis
siren = kbis[0]
orbis = extract_orbis(siren, driver)
print(orbis)

#appel tva
tva_full = orbis[3]
tva = tva_full.replace("FR","")

try :

  tva = extract_tva(tva, driver)
  print(tva)

except :

  print('une erreur est intervenue')

#appel insee
siret = orbis[2]
insee = api_insee_siret(siret)
print(insee)

##analyser le rib
position_rib = dossier.index(next(element for element in dossier if 'rib' in element))

try :

  rib = ocr_rib(dossier[position_rib])
  
except Exception as e:
  
  print(f'Une erreur est intervenue : {e}')
  path = dossier[position_rib]
  file_extension = os.path.splitext(path)[1]

  try :

    rib = extract_rib(dossier[position_rib])
    rename_file(dossier[position_rib], f'LINK_TO_TMP_FOLDER\\rib_{numero_reg}.{file_extension}')
    shutil.move(f'LINK_TO_TMP_FOLDER\\rib_{numero_reg}.{file_extension}', r"LINK_TO_TRAINING_DATA\rib" )

  except :

    rib = ["", "", "", "", "", "",""]
    rename_file(dossier[position_rib], f'LINK_TO_TMP_FOLDER\\rib_{numero_reg}.{file_extension}')
    shutil.move(f'LINK_TO_TMP_FOLDER\\rib_{numero_reg}.{file_extension}', r"LINK_TO_TRAINING_DATA\rib" )
    print('aucun rib trouvé')

print(rib)

#analyser checksum
try :

  IBAN(rib[0])

except InvalidChecksumDigits :

  rib = ["", "", "", "", "", "",""]
  print("iban invalid")

#fichier texte
fichier = fichier_texte(orbis, insee, rib, liste_pieces_jointes, '')

#verifications :
verifications_results = verifications_conforme(kbis, orbis, insee, tva)

#creer le fournisseur
if verifications_results == [] :

        print("les verifications sont ok")
        confirmation = input("Si vous voulez creer le fournisseur dans YB, tapez 'y'  :  ")
        if str(confirmation) == 'y' :
           
                creer_fournisseur_dans_yb = creer_fournisseur_youbuy(numero_reg, driver, orbis, insee)

        else :

          print('le fournisseur sera créé à posteriori')


        check = check_RVF_doublon(orbis[3], orbis[2])
        print(f"le resultat de l'analyse RVF est : {check[0]}")

        if check[0] == 'MAJRVF' :

            fichier_texte(orbis, insee, rib, tva, liste_pieces_jointes, check[1])
            print("vous pouvez utiiser le programme MAJ RVF")
            #creer_fournisseur_dans_sap(check[0])
            
        else :
          
            print('vous pouvez utiliser lancer programme AUTOSAP')
            #creer_fournisseur_dans_sap('AUTOSAP')

          
        deplacer_fichiers_vers_archive(f'{kbis[2]}')

else :

  print(f"fournisseur à verifier car : {verifications_results} ")


# 🚜 Call mass loading functions by siren searchs

* put all the siren in the list bellow

* Run the script

* download .txt generated files

In [ ]:
liste_siren = [793570383]

for siren in liste_siren:

    print(siren)

    try:

      orbis = extract_orbis(siren)
      print(orbis)

    except :

      print("une erreur est intervenue")
      continue

    # Appel à extract_tva

    if orbis[3] != "warning" :
      tva_full = orbis[3]
      tva = tva_full.replace("FR", "")
      tva = extract_tva(tva)
      print(tva)

    else :

      print("warning")
      continue

      # Appel à api_insee
    siret = orbis[2]
    insee = api_insee_siret(siret)
    print(insee)

    nom = orbis[0]
    siren = orbis[1]
    siret = orbis[2]
    tva = orbis[3]
    adresse = orbis[4]
    numero_rue = insee[3]
    if numero_rue == None :
      numero_rue = ''
    else :
      numero_rue = numero_rue
    rue = str(insee[4]) + " " + str(insee[5])
    postal_code = insee[6]
    city = insee[7]
    departement = postal_code[:2]
    nom_texte = nom.replace(" ", "_")
    # Enregistrer les valeurs dans un fichier texte
    with open(f"{nom_texte}.txt", "w") as f:
        f.write(f"{nom}\n")
        f.write(f"{rue}\n")
        f.write(f"{numero_rue}\n")
        f.write(f"{postal_code}\n")
        f.write(f"{city}\n")
        f.write(f"{departement}\n")
        f.write(f"{siren}\n")
        f.write(f"{siret}\n")
        f.write(f"{tva}\n")

      # Afficher le contenu du fichier texte dans la console
    with open(f'{nom_texte}.txt', 'r') as f:
        contenu = f.read()
        print(contenu)
